# For the first part of the challenge, please ingest and model the source data — try following the dbt modeling standards ⭐.
## Please include a document with information around:
  1. Preliminary data exploration
  2. Summary of your model architecture
  3. Lineage graphs
  4. Tips around macros, data validation, and documentation

# Step 1: Data Exploration using profiling

In [8]:
from ydata_profiling import ProfileReport
import pandas as pd
%matplotlib inline

In [12]:
df_acceptance = pd.read_csv("data/Globepay Acceptance Report - Globepay Acceptance Report.csv")
df_chargeback = pd.read_csv("data/Globepay Chargeback Report - Globepay Chargeback Report.csv")

In [13]:
df_acceptance.head()

,external_ref,status,source,ref,date_time,state,cvv_provided,amount,country,currency,rates
0,nCyEIIGaP5QQ2Wqb-_Rt8,True,GLOBALPAY,evt_1ESgcCOV7fY1ChY1MkZizZt,2019-01-01T00:00:00.000Z,DECLINED,False,1020.46,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
1,ptNiI9B7NNpK7VsMZx6zZ,True,GLOBALPAY,evt_1EBMGCEjEtianKMJaZXOx1y,2019-01-01T04:48:00.000Z,ACCEPTED,False,1582.57,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
2,1GS5BDKBnauCt-3nHPs6x,True,GLOBALPAY,evt_1EOALu0AZqEhZBn9ub_nbcE,2019-01-01T09:36:00.000Z,DECLINED,False,1653.07,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
3,Y_oecYz8GjBDaxmMqkkLQ,True,GLOBALPAY,evt_1ESHsWyaSPqE1mrmJ2B6rvs,2019-01-01T14:24:00.000Z,ACCEPTED,False,2228.17,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."
4,LlEs1TnKVa2xX6PH9Zsz0,True,GLOBALPAY,evt_1EjzPk13qLzmvDR_4hXuMMw,2019-01-01T19:12:00.000Z,ACCEPTED,False,2589.92,US,USD,"{""CAD"":1.415,""EUR"":0.926,""MXN"":25.103,""USD"":1,..."


In [14]:
df_chargeback.head()

,external_ref,status,source,chargeback
0,_-UTDS0dMlagMWEtQKMy-,True,GLOBALPAY,False
1,_0fqf75KiPa0iiviKCSsU,True,GLOBALPAY,False
2,_0H9Ecd_RctsLetId-p14,True,GLOBALPAY,False
3,_0wBrMzAzIhO3fQRWUcBH,True,GLOBALPAY,False
4,_25jZuGDrmwUEdAwKp1xV,True,GLOBALPAY,False


In [16]:
# profiling
report_acceptance = ProfileReport(df_acceptance, title="Acceptance Report")
report_chargeback = ProfileReport(df_chargeback, title="Chargeback Report")

report_acceptance.to_file("eda_reports/report_acceptance.html")
report_chargeback.to_file("eda_reports/report_chargeback.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 522.20it/s]


# Some first remarks after EDA about the DATA INGESTION
- The data ingestion will be performed below with vanilla python code, although for scalable use cases we would likely favor an ingestion tool like fivetran or airbyte.
- We will use postgres here, although in real life the data would likely be part of a big-data warehouse like e.g. Snowflake.
- The data could be ingested quicker with dbt seeds, but this is not the purpose of dbt seeds as the tables are dynamic in a source system.


# Some first remarks after EDA about the DATA CLEANING for the dbt staging layer
### General EDA remarks:
- No missing values in any column
- status and source columns are constant and can be ignored for this analysis
- external ref and ref columns are unique ids. external_ref of chargeback report and acceptance report can be used to merge the two files.
- dates provided between 01/01/2019 and 30/01/2029 (Q1 + Q2 2019). Also, we are dealing with 30 transactions a day. Column needs to be formatted.
- state var can be changed to a boolean "is_accepted"
- cvv_provided == True in <1% of cases
- Each country has the same number of transactions in this dataset
- AE charged in USD.
- Amounts in USD can be calculated from currency, amount, and rates
- chargeback rate of 4.1% overall
- The potentially relevant dimensions for this analysis are "date_time", "state", "cvv_provided", "amount_usd" (to be generated), "country", "chargeback".

### How to potentially enrich the data models:
- The data models could potentially be enriched, according to the Globepay API documentation. In fact, certain columns are requested by the API, but they aren't available in this dataset, e.g. payment method, expiry month, expiry year, (anonymized) card holder or company name 

### What this means for the staging layer:
1. Set up `_globepay__sources.yml` to ingest data into dbt
2. Set up `_globepay__models.yml` with column descriptions and basic test on unique columns
3. Implement the models, including column selection and data cleaning as described in the notebook cell above. 

# Some first remarks after EDA about the DATA MODELING for the dbt marts layer
- The two tables can be joined thanks to the `external_refs` column.
- The column descriptions can be extracted either from the `Globe API Specification.html` file or inferred.
- The column descriptions can be inserted into the `dbt_warehouse/models/marts/payments/_payment__models.yml` file.

# What will the data lineage graph look like?
### 1. Ingestion: The files will be ingested with our vanilla script implemented below
  - `data/Globepay Acceptance Report - Globepay Acceptance Report.csv` -> `prod_dwh.raw.src_globepay__acceptance_report`
  - `data/Globepay Chargeback Report - Globepay Chargeback Report.csv` -> `prod_dwh.raw.src_globepay__chargeback_report`
### 2. Sourcing: The ingested files will be sourced as a dbt model thanks to the file `_globepay__sources.yml`
  - `prod_dwh.raw.src_globepay__acceptance_report` -> `src_globepay__acceptance_report`
  - `prod_dwh.raw.src_globepay__chargeback_report` -> `src_globepay__chargeback_report`
### 3. Staging: The sourced files will be be described thanks to the file `staging/globepay/_globepay__models` and cleaned as a dbt model thanks to the following transformation
  - `src_globepay__acceptance_report` -> `stg_globepay__acceptance_report`
  - `src_globepay__chargeback_report` -> `stg_globepay__chargeback_report`
### 3. Intermediate: Because of low model complexity, we will not use `intermediate` models.
### 4. Marts: The staged files will be be described thanks to the file `marts/payments/_payment__models` and transformed as a dbt model thanks to the following transformation
  - `stg_globepay__acceptance_report` + `stg_globepay__chargeback_report` -> `acceptance_rate_by_day`
  - `stg_globepay__acceptance_report` + `stg_globepay__chargeback_report` -> `amount_declined_over_25m_by_country`
  - `stg_globepay__acceptance_report` + `stg_globepay__chargeback_report` -> `transactions_missing_chargeback`


# Ingestion

In [24]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_docker_host = os.getenv('DB_DOCKER_HOST')
db_port = int(os.getenv('DB_PORT'))
db_name = os.getenv('DB_NAME')
db_schema = os.getenv('DB_SCHEMA')


db_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)


df_acceptance.to_sql(name = "src_globepay__acceptance_report", con=engine, schema=db_schema, if_exists='replace', index=False) 
df_chargeback.to_sql(name = "src_globepay__chargeback_report", con=engine, schema=db_schema, if_exists='replace', index=False) 
print("Data successfully written to PostgreSQL!")


Data successfully written to PostgreSQL!
